# Giriş İşlemleri

In [ ]:
%%capture
!pip install kagglehub pandas

In [ ]:
import kagglehub
import pandas as pd
import os
import zipfile

# Veri Kümesi Okuma

## Veri kümesi okuma fonksiyonu

In [ ]:
def veri_kumesi_oku(veri_kumesi_adi: str) -> pd.DataFrame:
    """
    Kaggle veri kümesini indirir ve döndürür.
    
    Args:
        veri_kumesi_adi (str): Kaggle veri kümesinin yolu.
        gecici_yol (str): Geçici dosya yolu.
    Returns:
        pd.DataFrame: İndirilen veri kümesi.
    """
    # Kaggle veri kümesini indir
    gecici_yol = os.path.join(kagglehub.dataset_download(veri_kumesi_adi), "TurkishSMSCollection.csv")
    return pd.read_csv(gecici_yol, sep=";", encoding="utf-8")

## Veri kümesini oku

In [ ]:
# Veri kümesi bilgileri
dataset_name = "onurkarasoy/turkish-sms-collection"

veri_kumesi = veri_kumesi_oku(dataset_name)
veri_kumesi.head(10)  # İlk 10 satırı göster

## Veri kümesinin gereksiz kolonlarını silme

In [ ]:
tohum_degeri = 571
# gereksiz sütunları kaldır
veri_kumesi = veri_kumesi[['Message', 'Group']]
# veri kümesini karıştır
veri_kumesi = veri_kumesi.sample(frac=1, random_state=tohum_degeri).reset_index(drop=True)
veri_kumesi.head(10)  # İlk 10 satırı göster

## Veri kümesini eğitim ve sınama olarak parçalama